In [4]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import StandardScaler

def load_images_from_folder(data_dir):
    images = []
    labels = []
    
    # Loop through each class subdirectory
    for class_label in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_label)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                try:
                    # Load the image and convert to grayscale
                    img = Image.open(img_path).convert('L')  # Convert to grayscale
                    img = img.resize((28, 28))  # Resize to 28x28 if needed
                    img_array = np.array(img).flatten()  # Flatten the image to a vector
                    
                    images.append(img_array)
                    labels.append(int(class_label))  # Use folder name as label
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
    
    return np.array(images), np.array(labels)

# Load training and test data
train_dir = r"C:\Users\FAIZ SIDDIQUI\Favorites\Downloads\MNISToriginal\mnist_png\train"
test_dir = r"C:\Users\FAIZ SIDDIQUI\Favorites\Downloads\MNISToriginal\mnist_png\test"

X_train, y_train = load_images_from_folder(train_dir)
X_test, y_test = load_images_from_folder(test_dir)

# Normalize features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
class LinearSVM:
    def __init__(self, learning_rate=0.001, C=1.0, n_iters=1000):
        self.lr = learning_rate
        self.C = C  # Regularization parameter
        self.n_iters = n_iters
        self.w = None
        self.b = None
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        # Convert labels from {0,1} to {-1,1}
        y_ = np.where(y <= 0, -1, 1)
        
        # Initialize weights and bias
        self.w = np.zeros(n_features)
        self.b = 0
        
        # Gradient descent
        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    # No misclassification: Only regularization term applies
                    self.w -= self.lr * (2 * self.C * self.w)
                else:
                    # Misclassification: Apply hinge loss and regularization
                    self.w -= self.lr * (2 * self.C * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]
    
    def predict(self, X):
        return np.sign(np.dot(X, self.w) - self.b)


In [12]:
class QuadraticSVM:
    def __init__(self, learning_rate=0.01, C=0.1, n_iters=10):
        self.lr = learning_rate
        self.C = C  # Regularization parameter
        self.n_iters = n_iters
        self.alpha = None
        self.b = 0
        self.X_train = None
        self.y_train = None

    def polynomial_kernel(self, x1, x2):
        return (np.dot(x1, x2) + 1) ** 2  # Polynomial kernel of degree 2

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.alpha = np.zeros(n_samples)
        self.X_train = X
        self.y_train = np.where(y <= 0, -1, 1)

        for _ in range(self.n_iters):
            for i in range(n_samples):
                # Gradient calculation
                gradient = self.C * (1 - self.y_train[i] * np.sum(
                    [self.alpha[j] * self.y_train[j] * self.polynomial_kernel(X[i], X[j]) 
                     for j in range(n_samples)]
                ))
                self.alpha[i] += self.lr * gradient
        # Bias calculation
        self.b = np.mean([y_i - np.sum([self.alpha[j] * self.y_train[j] * self.polynomial_kernel(X[i], X[j]) 
                      for j in range(n_samples)]) for i, y_i in enumerate(self.y_train)])

    def predict(self, X):
        return np.sign(np.sum([self.alpha[i] * self.y_train[i] * self.polynomial_kernel(X, self.X_train[i]) 
                               for i in range(len(self.X_train))]) - self.b)


In [9]:
# Train Linear SVM
linear_svm = LinearSVM(learning_rate=0.001, C=1.0, n_iters=1000)
linear_svm.fit(X_train_scaled, y_train)

# Evaluate Linear SVM on Test Data
y_pred_linear = linear_svm.predict(X_test_scaled)

# Calculate performance metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy_linear = accuracy_score(y_test, y_pred_linear)
precision_linear = precision_score(y_test, y_pred_linear, average='macro')  # Average for multiclass
recall_linear = recall_score(y_test, y_pred_linear, average='macro')
f1_linear = f1_score(y_test, y_pred_linear, average='macro')

print(f"Linear SVM: Accuracy={accuracy_linear}, Precision={precision_linear}, Recall={recall_linear}, F1={f1_linear}")


Linear SVM: Accuracy=0.1135, Precision=0.01135, Recall=0.1, F1=0.0203861697350696


c:\Users\FAIZ SIDDIQUI\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
# Train Quadratic SVM
quadratic_svm = QuadraticSVM(learning_rate=0.01, C=0.1, n_iters=10)
quadratic_svm.fit(X_train_scaled, y_train)

# Evaluate Quadratic SVM on Test Data
y_pred_quadratic = [quadratic_svm.predict(x) for x in X_test_scaled]

# Calculate performance metrics
accuracy_quad = accuracy_score(y_test, y_pred_quadratic)
precision_quad = precision_score(y_test, y_pred_quadratic, average='macro')
recall_quad = recall_score(y_test, y_pred_quadratic, average='macro')
f1_quad = f1_score(y_test, y_pred_quadratic, average='macro')

print(f"Quadratic SVM: Accuracy={accuracy_quad}, Precision={precision_quad}, Recall={recall_quad}, F1={f1_quad}")

C:\Users\FAIZ SIDDIQUI\AppData\Local\Temp\ipykernel_9512\3255259921.py:24: RuntimeWarning: overflow encountered in scalar multiply
  [self.alpha[j] * self.y_train[j] * self.polynomial_kernel(X[i], X[j])
